In [10]:
import os
import MDAnalysis as mda
import prolif as plf
from MDAnalysis.topology import guessers
from tqdm.auto import tqdm
import pandas as pd

In [2]:
SYSTEMS = {
    "WT": ("WT.gro", "md_200ns_fit.xtc", "WT_frame0.pdb"),
    "P48L_H83Y": ("P48L_H83Y.gro", "md_p48l_h83y.xtc", "P48L_H83Y_frame0.pdb"),
    "R131P_G55C": ("R131P_G55C.gro", "md_r131p_g55c.xtc", "R131P_G55C_frame0.pdb"),
}

###### We generate frame0.pdb to obtain a topology that exactly matches the trajectory.
###### The PDB file is extracted from the first frame of the XTC trajectory, ensuring it contains the same atoms and atom ordering as the trajectory. It is then used as a reliable topology for ProLIF analysis, independently of any issues with the .gro file.

In [3]:
# --- utilities ---
def ensure_elements(u: mda.Universe):
    if not hasattr(u.atoms, "elements"):
        u.add_TopologyAttr("elements", guessers.guess_types(u.atoms.names))

def make_frame0_pdb_from_gro(gro: str, out_pdb: str):
    u0 = mda.Universe(gro)  # topology only
    ensure_elements(u0)
    u0.atoms.write(out_pdb)

In [4]:
# --- ensure that frame0 PDB is available for all systems ---
for sysname, (gro, xtc, out_pdb) in SYSTEMS.items():
    if not os.path.exists(out_pdb):
        print(f"[{sysname}] writing {out_pdb} from {gro}")
        make_frame0_pdb_from_gro(gro, out_pdb)
    else:
        print(f"[{sysname}] exists: {out_pdb}")

[WT] exists: WT_frame0.pdb
[P48L_H83Y] exists: P48L_H83Y_frame0.pdb
[R131P_G55C] exists: R131P_G55C_frame0.pdb


In [5]:
# --- configure the fingerprint ---
fp = plf.Fingerprint([
    "HBDonor",
    "HBAcceptor",
    "Anionic",
    "Cationic",
    "PiCation",
    "Hydrophobic",
    "VdWContact",
])

In [ ]:
# Compute an intraprotein fingerprint with frame subsampling and save the results
# Output: *_fingerprint.pkl files containing intraprotein interactions

results = {}

target_frames = 4000 # can be changed 

for sysname, (gro, xtc, pdb) in SYSTEMS.items():
    print(f"\n=== {sysname} ===")

    u = mda.Universe(pdb, xtc)
    ensure_elements(u)

    protein = u.select_atoms("protein")
    protein.guess_bonds()

    n_frames = len(u.trajectory)
    step = max(1, n_frames // target_frames)
    used_frames = (n_frames + step - 1) // step
    print(f"Total frames: {n_frames} → using {used_frames} frames (step={step})")

    fp_sys = plf.Fingerprint(fp.interactions)
    fp_sys.run(u.trajectory[::step], protein, protein, progress=True, n_jobs=1)

    pkl_name = f"{sysname}_fingerprint.pkl"
    fp_sys.to_pickle(pkl_name)
    print(f"Saved: {pkl_name}")

    results[sysname] = fp_sys

results

In [15]:
def unique_interaction_stats(fp_sys):
    """
    Removes A–A interactions and duplicate A–B / B–A pairs.
    Returns:
      - total_unique: total number of unique interactions
      - per_type: number of unique interactions per interaction type
      - per_frame: average number of unique interactions per frame
    """
    df = fp_sys.to_dataframe()

    # remove A–A
    df = df.loc[:, df.columns.get_level_values(0) != df.columns.get_level_values(1)]

    # remove duplicate A–B / B–A pairs
    cols = df.columns.to_frame(index=False)
    cols["pair"] = cols.apply(
        lambda r: tuple(sorted((r[0], r[1]))),
        axis=1
    )
    cols["pair_interaction"] = list(zip(cols["pair"], cols["interaction"]))
    mask = ~cols["pair_interaction"].duplicated()

    df_unique = df.loc[:, mask.values]

    # statistics
    total_unique = int(df_unique.values.sum())
    per_type = df_unique.sum().groupby(level="interaction").sum()
    per_frame = total_unique / df_unique.shape[0]

    return total_unique, per_frame, per_type

In [16]:
summary = {}
per_type_all = {}

for name, fp_sys in results.items():
    total, per_frame, per_type = unique_interaction_stats(fp_sys)

    summary[name] = {
        "total_unique_interactions": total,
        "unique_interactions_per_frame": per_frame,
    }
    per_type_all[name] = per_type

# table with overall counts
summary_df = pd.DataFrame(summary).T
summary_df

# summary_df — total number and average number of contacts per frame
# per_type_all — breakdown by interaction types

/tmp/ipykernel_4615/2296032134.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lambda r: tuple(sorted((r[0], r[1]))),
/tmp/ipykernel_4615/2296032134.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lambda r: tuple(sorted((r[0], r[1]))),
/tmp/ipykernel_4615/2296032134.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lambda r: tuple(sorted((r[0], r[1]))),


,total_unique_interactions,unique_interactions_per_frame
WT,2948416.0,736.919770
P48L_H83Y,2931819.0,732.771557
R131P_G55C,2870260.0,717.385654


In [17]:
per_type_df = (
    pd.DataFrame(per_type_all)
    .fillna(0)
    .astype(int)
    .sort_index()
)

per_type_df

# per_type_df table: for each system (WT, mutants)
# Number of unique intraprotein interactions grouped by contact type

,WT,P48L_H83Y,R131P_G55C
interaction,,,
Anionic,57151,57107,52689
Cationic,57151,57107,52689
HBAcceptor,397762,403444,392518
HBDonor,389398,390589,374899
Hydrophobic,569589,551157,529601
PiCation,4367,4146,3236
VdWContact,1472998,1468269,1464628
